NSP return 0 if the sentence we are evaluating is the next sentence, 1 otherwise

In [11]:
from transformers import BertTokenizer, BertForPreTraining
import torch

Inizialize our model

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Where text is sentence a and text2 is sentence b, b is after sentence a, so we expect 0 as NSP return

In [13]:
text = ("After Abraham Lincoln won the November 1860 presidential [MASK] on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy.")
text2 = ("War broke out in April 1861 when secessionist forces [MASK] Fort "
         "Sumter in South Carolina, just over a month after Lincoln's "
         "inauguration.")

So now we have to take 3 different steps:
- tokenization
- create a calssification label so we can train the model
- calculate the loss

In [18]:
#Tokenization
inputs = tokenizer(text, text2, return_tensors='pt')
inputs
#there is a "102" which is a separator token between sentence a and b
#also in tensor all the 0 rapresent sentence a and all the 1 sentence b

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,   103,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,   102,  2162,  3631,  2041,  1999,  2258,  6863,
          2043, 22965,  2923,  2749,   103,  3481,  7680,  3334,  1999,  2148,
          3792,  1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055,
         17331,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [17]:
inputs.keys() #now we need all of them

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [20]:
#create label tensor
labels = torch.LongTensor([0]) #we need to pass a list containing a single value (0 - if is the next sentence, 1 - if isn't the next sentence)
#in our case the two sencences are supposed to be together so we pass 0
labels

tensor([0])

In [28]:
outputs = model(**inputs, labels = labels)
outputs.keys()
#the loss will only be output if we pass the labels

In [25]:
outputs.seq_relationship_logits
#where 6.0843 is the actiation for "is the next sentence" and -5.6813 is 
# the activation for "is not the next sentence"

tensor([[ 6.0843, -5.6813]], grad_fn=<AddmmBackward0>)

In [26]:
torch.argmax(outputs.seq_relationship_logits)
#we get 0 which mean that is the next sentence

tensor(0)